# Transformadores personalizados

Este notebook es una adaptación del [original de *Aurélien Gerón*](https://github.com/ageron/handson-ml3/blob/main/02_end_to_end_machine_learning_project.ipynb), de su libro: [Hands-On Machine Learning with Scikit-Learn, Keras, and TensorFlow, 3rd Edition. Aurélien Géron](https://www.oreilly.com/library/view/hands-on-machine-learning/9781098125967/)

## Pasos previos

In [9]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

housing = pd.read_csv("./data/housing.csv") 

# Generación de conjuntos de entrenamiento y prueba mediante muestreo estratificado por ingreso medio
train_set, test_set = train_test_split(housing, test_size=0.2,
    stratify=pd.cut(housing["median_income"], bins=[0., 1.5, 3.0, 4.5, 6., np.inf], labels=[1, 2, 3, 4, 5]),
    random_state=42
    )

housing = train_set.drop("median_house_value", axis=1) # Eliminamos la columna de la variable dependiente
housing_labels = train_set["median_house_value"].copy() # Guardamos la variable dependiente (etiquetas)

## Creación de transformadores personalizados

Para transformaciones que no requieran entrenamiento, puede definirse simplemente una función que recibe un array de NumPy y devuelve otro transformado y pasársela a `FunctionTransformer` para crear un transformador personalizado. Estos transformadores permitirán crear objetos que se comporten como los de la librería `sklearn` y que puedan ser utilizados en los *pipelines* de la misma. Por ejemplo, para transformaciones logarítmicas

In [10]:
from sklearn.preprocessing import FunctionTransformer

log_transformer = FunctionTransformer(np.log, inverse_func=np.exp)
log_pop = log_transformer.transform(housing[["population"]])

o para combinar *features*:

In [11]:
def column_ratio(X): # Custom transformer to compute the ratio of two columns
    return X[:, [0]] / X[:, [1]]

ratio_transformer = FunctionTransformer(column_ratio)
    
housing["rooms_per_household"] = ratio_transformer.fit_transform(housing[['total_rooms', 'households']].values)
housing[['rooms_per_household', 'total_rooms', 'households']].head()

,rooms_per_household,total_rooms,households
12655,5.485836,3873.0,706.0
15502,6.927083,5320.0,768.0
2908,5.393333,1618.0,300.0
14053,3.886128,1877.0,483.0
20496,6.096552,3536.0,580.0


El mismo ejemplo de antes, se puede definir de forma más compacta usando una lambda (una función anónima):

In [12]:
ratio_transformer = FunctionTransformer(lambda X: X[:, [0]] / X[:, [1]])
housing["rooms_per_household"] = ratio_transformer.transform(housing[['total_rooms', 'households']].values)
housing[['rooms_per_household', 'total_rooms', 'households']].head()

,rooms_per_household,total_rooms,households
12655,5.485836,3873.0,706.0
15502,6.927083,5320.0,768.0
2908,5.393333,1618.0,300.0
14053,3.886128,1877.0,483.0
20496,6.096552,3536.0,580.0


Cuando nuestra transformación requiere entrenamiento, podemos crear un transformador que tenga un método `fit` en el que se aprendan los parámetros necesarios y un método `transform` que aplique la transformación. Un transformador personalizado debe heredar de `BaseEstimator` (del que hereda los métodos `get_params` y `set_params`, necesarios para ajustar los parámetros de la transformación) y de `TransformerMixin` (que proporciona el método `fit_transform`). 

Por ejemplo, definiendo un transformador que se comporte como `StandardScaler`:

In [13]:
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.utils.validation import check_array, check_is_fitted

class StandardScalerClone(BaseEstimator, TransformerMixin):
    def __init__(self, with_mean=True):  # no *args or **kwargs!
        self.with_mean = with_mean

    def fit(self, X, y=None):  # y is required even though we don't use it
        X = check_array(X)  # checks that X is an array with finite float values
        self.mean_ = X.mean(axis=0)
        self.scale_ = X.std(axis=0)
        self.n_features_in_ = X.shape[1]  # every estimator stores this in fit()
        return self  # always return self!

    def transform(self, X):
        check_is_fitted(self)  # looks for learned attributes (with trailing _)
        X = check_array(X)
        assert self.n_features_in_ == X.shape[1]
        if self.with_mean:
            X = X - self.mean_
        return X / self.scale_

In [14]:
# Ejemplo de uso de un transformador personalizado
scaler = StandardScalerClone()
scaler.fit(housing[["total_rooms"]])
scaler.transform(housing[["total_rooms"]])

array([[ 0.58477745],
       [ 1.26146668],
       [-0.46977281],
       ...,
       [-0.89580177],
       [ 0.2490049 ],
       [-0.72183605]])